In [465]:
import pandas as pd

In [466]:
result = pd.read_csv('./total_padded_encoded_df.csv')
data = pd.DataFrame(result)
data.head()

,Unnamed: 0,genre,1,2,3,4,5,6,7,8,...,843,844,845,846,847,848,849,850,851,852
0,0,1,0,2,3,0,0,4,0,5,...,1,1,1,1,1,1,1,1,1,1
1,1,1,31,41,42,43,44,45,0,46,...,1,1,1,1,1,1,1,1,1,1
2,2,1,138,139,140,0,0,0,141,0,...,1,1,1,1,1,1,1,1,1,1
3,3,1,117,0,203,0,204,205,206,0,...,1,1,1,1,1,1,1,1,1,1
4,4,1,284,0,285,286,287,0,288,288,...,1,1,1,1,1,1,1,1,1,1


In [467]:
data['genre'] = data['genre'].replace({1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7})

# feature, label 분리

In [468]:
feature_df = data.iloc[:, 2:]
label_df = data[['genre']]
print(feature_df.shape, label_df.shape)

(772, 852) (772, 1)


In [469]:
feature_df.head()

,1,2,3,4,5,6,7,8,9,10,...,843,844,845,846,847,848,849,850,851,852
0,0,2,3,0,0,4,0,5,6,7,...,1,1,1,1,1,1,1,1,1,1
1,31,41,42,43,44,45,0,46,0,47,...,1,1,1,1,1,1,1,1,1,1
2,138,139,140,0,0,0,141,0,143,0,...,1,1,1,1,1,1,1,1,1,1
3,117,0,203,0,204,205,206,0,118,207,...,1,1,1,1,1,1,1,1,1,1
4,284,0,285,286,287,0,288,288,289,290,...,1,1,1,1,1,1,1,1,1,1


In [470]:
label_df.head()

,genre
0,0
1,0
2,0
3,0
4,0


In [471]:
# label_df = label_df.replace({1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7})

In [472]:
label_df['genre'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [473]:
label_df.head()

,genre
0,0
1,0
2,0
3,0
4,0


# DataSet 생성, Tensor 변환, DataLoader 생성

In [474]:
from torch.utils.data import Dataset, DataLoader, random_split

import torch
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.x_data = data.iloc[:, 2:].values
        self.y_data = data[['genre']].values
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,idx):
        x = torch.LongTensor(self.x_data[idx])
        y = torch.LongTensor(self.y_data[idx])
        return x, y

In [475]:
DS = CustomDataset()

num = int(len(DS) * 0.9)
print(num)

694


In [476]:
# 학습용과 테스트용으로 분리
train_ds, test_ds = random_split(DS, [num, len(DS) - num])

In [477]:
# 데이터 로더 생성
BATCH = 20
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle = True)
test_loader = DataLoader(test_ds, batch_size = BATCH, shuffle = True)

# 모델 설계

In [478]:
import torch.nn as nn
class music_rnn(nn.Module):
    def __init__(self, VOCAB_SIZE, EMBED_DIM, HIDDEN_SIZE, NUM_CLASS):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=EMBED_DIM)
        self.rnn = nn.RNN(input_size=EMBED_DIM, hidden_size=HIDDEN_SIZE, batch_first = True)
        # 학습 모델을 RNN 대신에 LSTM을 사용해볼수도 있겠다
        # RNN에서 양방향 학습 파라미터를 설정해볼 수 도 있겠다.
        
        # 활성화함수 => 다중 분류니까
        # self.fc = nn.Softmax(HIDDEN_SIZE, NUM_CLASS) # 이렇게 쓰는게 아닌가?
        self.fc = nn.Linear(HIDDEN_SIZE, NUM_CLASS) # 다중 분류인데 이게 되나?
        
        self.init_weights() # 이건 왜 쓴거지?
        self.dropout = nn.Dropout()  # 혹시 모를 과대적합을 위해서
        
    # 가중치 초기화
    def init_weights(self):
        range = 0.5
        self.embedding.weight.data.uniform_(-range, range)
        # self.fc.weight.data.uniform_(-range, range)
        # self.fc.bias.data.zero_()
        
    # 순방향 학습 진행
    def forward(self, text):
        embed = self.embedding(text)
        print(f"embed shape : {embed.shape}")
        output, hidden = self.rnn(embed)
        print(f"output shape : {output.shape}")
        print(f"hidden shape : {hidden.shape}")
        output=output[:,-1,:]
        result = self.fc(output)
        print('[ class에서 forward 진행 후 결과 ]')
        
        print(f"result shape : {result.shape}")# 이 층을 왜 통과시켜줘야하는거지? ;;
        # result = F.softmax(self.fc(output), dim=1)
        return result

In [479]:
# 학습용 변수들 지정
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F
VOCAB_SIZE = 13714
EMBED_DIM = 50
HIDDEN_SIZE = 10
NUM_CLASS = 8
model = music_rnn(VOCAB_SIZE, EMBED_DIM, HIDDEN_SIZE, NUM_CLASS)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(params = model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [480]:
model

music_rnn(
  (embedding): Embedding(13714, 50)
  (rnn): RNN(50, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=8, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

# 함수 지정 (train, test)

In [481]:
import torchmetrics.functional.classification as metrics
import numpy as np
def training(DATALOADER):
    model.train()
    output_list = [[], [], [], [], []]
    # loss, acc, precision, recall, f1_score
    for (feature, label) in DATALOADER:
        label2 = F.one_hot(label)
        label2 = label2.squeeze(dim=1)
        print(f"label shape : {label.shape}")
        # label = label.expand(-1,8)
        
        feature = feature.to(device)
        label2 = label2.to(device)

        pred = model(feature)
        print('-'*50)
        print('[ 학습 함수 안에서 결과 ]')
        print(f"feature shape : {feature.shape}")
        # print(f"label shape : {label.shape}")
        print(f"label2 shape : {label2.shape}")
        print(f"pred shape : {pred.shape}")
        print('-'*50)
        
        # pred = label.view(-1,8).float()
        # print(feature, label, pred, sep = '\n\n', end =  '\n\n')
        # print('-'*50)
        # print(f"feature.shape - {feature.shape}  label.shape - {label.shape}  pred.shape - {pred.shape}")
        
        # 손실 검정
        loss = loss_fn(pred, label2.float()) 
        output_list[0].append(loss.item())
        output_list[1].append(metrics.accuracy(pred, label2, task = 'binary').item())
        output_list[2].append(metrics.precision(pred, label2, task = 'binary').item())
        output_list[3].append(metrics.recall(pred, label2, task = 'binary').item())
        output_list[4].append(metrics.f1_score(pred, label2, task = 'binary').item())
        
        # 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return output_list
training(train_loader)

label shape : torch.Size([20, 1])
embed shape : torch.Size([20, 852, 50])
output shape : torch.Size([20, 852, 10])
hidden shape : torch.Size([1, 20, 10])
[ class에서 forward 진행 후 결과 ]
result shape : torch.Size([20, 8])
--------------------------------------------------
[ 학습 함수 안에서 결과 ]
feature shape : torch.Size([20, 852])
label2 shape : torch.Size([20, 8])
pred shape : torch.Size([20, 8])
--------------------------------------------------
label shape : torch.Size([20, 1])
embed shape : torch.Size([20, 852, 50])
output shape : torch.Size([20, 852, 10])
hidden shape : torch.Size([1, 20, 10])
[ class에서 forward 진행 후 결과 ]
result shape : torch.Size([20, 8])
--------------------------------------------------
[ 학습 함수 안에서 결과 ]
feature shape : torch.Size([20, 852])
label2 shape : torch.Size([20, 8])
pred shape : torch.Size([20, 8])
--------------------------------------------------
label shape : torch.Size([20, 1])
embed shape : torch.Size([20, 852, 50])
output shape : torch.Size([20, 852, 10])
h

RuntimeError: 0D or 1D target tensor expected, multi-target not supported